In [1]:
using DynamicPolynomials
using TSLTSSOS
using MAT
using LinearAlgebra
using Printf

In [2]:
@polyvar temp1[1:4, 1:4]
@polyvar temp2[1:4, 1:4]
temp_p = temp2 * 1.1 + temp1 * 1.1

function skew(q)
    qx = temp_p[1:3, 1:3] * 0
    qx[1, 2] = -q[3]
    qx[2, 1] =  q[3]

    qx[1, 3] =  q[2]
    qx[3, 1] = -q[2]

    qx[2, 3] = -q[1]
    qx[3, 2] =  q[1]

    return qx
end

function LeftMultiplication(q)
    Lq = temp_p * 0
    Lq[1, 1]   =  q[1]
    Lq[1, 2:4] = -q[2:4]
    Lq[2:4, 1] = q[2:4]
    Lq[2:4, 2:4] = q[1] * Matrix(1.0I, 3, 3) + skew(q[2:4])
    return Lq
end

function RightMultiplication(q)
    Rq = temp_p * 0
    Rq[1, 1]   =  q[1]
    Rq[1, 2:4] = -q[2:4]
    Rq[2:4, 1] = q[2:4]
    Rq[2:4, 2:4] = q[1] * Matrix(1.0I, 3, 3) - skew(q[2:4])
    return Rq
end

V = temp_p[1:3, :] * 0
V[1:3, 2:4] = Matrix(1.0I, 3, 3)
T = temp_p * 0
T[2:4, 2:4] = -Matrix(1.0I, 3, 3)
T

4×4 Matrix{Polynomial{true, Float64}}:
 0.0  0.0   0.0   0.0
 0.0  -1.0  0.0   0.0
 0.0  0.0   -1.0  0.0
 0.0  0.0   0.0   -1.0

In [3]:
N = 20
@polyvar q[1:N+1, 1:4]
@polyvar w[1:N+1, 1:4]

skew(q[1, :])
LeftMultiplication(q[1, :]) - RightMultiplication(q[1, :])


4×4 Matrix{Polynomial{true, Float64}}:
 0.0  0.0       0.0       0.0
 0.0  0.0       -2.0q₁₋₄  2.0q₁₋₃
 0.0  2.0q₁₋₄   0.0       -2.0q₁₋₂
 0.0  -2.0q₁₋₃  2.0q₁₋₂   0.0

In [19]:
Jb = Matrix(1.0I, 3, 3)
Jb[1,1] = 1.0 / 10
Jb[2,2] = 2.0 / 10
Jb[3,3] = 1.0 / 10
m = 0.5

G = temp_p[1:3, 1] * 0
G[3] = 1

1

In [5]:
qk   = q[1, :]
qkp1 = q[2, :]

Mk = Jb * qk[1] * qk[2:4] + skew(qk[2:4]) * Jb * qk[2:4]
Mkp1 = Jb * qkp1[1] * qkp1[2:4] + skew(qkp1[2:4]) * Jb * qkp1[2:4]

Mkp1 - Mk

3-element Vector{Polynomial{true, Float64}}:
 -0.1q₁₋₁q₁₋₂ + 0.1q₂₋₁q₂₋₂ + 0.1q₁₋₃q₁₋₄ - 0.1q₂₋₃q₂₋₄
 -0.2q₁₋₁q₁₋₃ + 0.2q₂₋₁q₂₋₃
 -0.1q₁₋₁q₁₋₄ + 0.1q₂₋₁q₂₋₄ - 0.1q₁₋₂q₁₋₃ + 0.1q₂₋₂q₂₋₃

In [64]:
# Rotation only
N = 20
dt = 0.125

@polyvar q[1:N+1, 1:4]
@polyvar w[1:N+1, 1:4]
@polyvar tau[1:N, 1:3]

eq = [temp_p[1,1] * 0]
ineq = []

eq[1] = q[1, 1] - 1.0
append!(eq, [q[1, 2] - 0.0])
append!(eq, [q[1, 3] - 0.0])
append!(eq, [q[1, 4] - 0.0])

append!(eq, [w[1, 1] - 2 / dt])
append!(eq, [w[1, 2] - 0.0])
append!(eq, [w[1, 3] - 0.0])
append!(eq, [w[1, 4] - 0.0])

var = append!(reshape(q, 4 * (N+1)), reshape(w, 4*(N+1)), reshape(tau, 3 * N))

Cost = 0
for k = 1:N

    qk = q[k, 1:4]
    qkp1 = q[k+1, 1:4]

    wk = w[k, 1:4]
    wkp1 = w[k+1, 1:4]

    tauk = tau[k, 1:3]

    Mk = Jb * wk[1] * wk[2:4] + skew(wk[2:4]) * Jb * wk[2:4]
    Mkp1 = Jb * wkp1[1] * wkp1[2:4] + skew(wkp1[2:4]) * Jb * wkp1[2:4]

    append!(eq, Mkp1 - Mk - 2 * tauk)
    append!(eq, qkp1 - dt / 2 * LeftMultiplication(qk) * wk)

    append!(eq, [qkp1[1]^2 + qkp1[2]^2 + qkp1[3]^2 + qkp1[4]^2 - 1])
    append!(eq, [(wkp1[1]^2 + wkp1[2]^2 + wkp1[3]^2 + wkp1[4]^2) * dt^2 - 4])
    
    append!(ineq, [wkp1[1]])
    append!(ineq, [qkp1[1]])

    Cost = Cost + (tauk[1]^2 + tauk[2]^2 + tauk[3]^2) * 0.01
    Cost = Cost + ((wk[1] - 1)^2 + wk[2]^2 + wk[3]^2 + wk[4]^2)
    Cost = Cost + (qk[1]^2 + (qk[2] - 1)^2 + qk[3]^2 + qk[4]^2)


end

wkp1 = w[end, :]

Cost = Cost + ((wkp1[1] - 1)^2 + wkp1[2]^2 + wkp1[3]^2 + wkp1[4]^2)
Cost = Cost + (qkp1[1]^2 + (qkp1[2] - 1)^2 + qkp1[3]^2 + qkp1[4]^2)

pop = append!([Cost], ineq, eq)
eq_len = length(eq)

order = 1
# opt,sol,data =cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS = "block", solution=true, QUIET=false, MomentOne=false)
opt,sol,data =cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS = "block" , solution=true, QUIET=false, MomentOne=false)

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2022
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[4, 1, 3]
[160, 64, 2]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.0010101 seconds. The maximal size of cliques is 4.
Starting to compute the block structure...
Obtained the block structure in 0.009154 seconds. The maximal size of blocks is 5.
Assembling the SDP...
There are 882 affine constraints.
SDP assembling time: 0.004299 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 882             
  Cones                  : 0               
  Scalar variables       : 229             
  Matrix variables       :

(4737.543415593808, [0.9999999204478611, 0.9999996066030024, 0.954254011574492, 0.8411596434757946, 0.7060688673295643, 0.5755027416493457, 0.4608247058237058, 0.36508100179123837, 0.2874196548168741, 0.22549229386208716  …  -2.038504770039333e-18, 1.1810832101509145e-18, 0.0, 9.157926925726512e-19, -1.060068829820371e-18, -6.263341817413688e-19, 0.0, 6.666944127649452e-19, 6.930338575127257e-19, 0.0], TSLTSSOS.mcpop_data(228, 0, 228, 188, Vector{Vector{UInt16}}[[[], [0x0001, 0x0001], [0x0002, 0x0002], [0x0003, 0x0003], [0x0004, 0x0004], [0x0005, 0x0005], [0x0006, 0x0006], [0x0007, 0x0007], [0x0008, 0x0008], [0x0009, 0x0009]  …  [0x00db, 0x00db], [0x00dc, 0x00dc], [0x00dd, 0x00dd], [0x00de, 0x00de], [0x00df, 0x00df], [0x00e0, 0x00e0], [0x00e1, 0x00e1], [0x00e2, 0x00e2], [0x00e3, 0x00e3], [0x00e4, 0x00e4]], [[0x0056]], [[0x0002]], [[0x0057]], [[0x0003]], [[0x0058]], [[0x0004]], [[0x0059]], [[0x0005]], [[0x005a]]  …  [[0x0068, 0x0068], [0x007d, 0x007d], [0x0092, 0x0092], [0x00a7, 0x00a7]

In [48]:
# Rotation + position
N = 20
dt = 0.1

@polyvar q[1:N+1, 1:4]
@polyvar w[1:N+1, 1:4]

@polyvar p[1:N+1, 1:3]
@polyvar v[1:N+1, 1:3]

@polyvar tau[1:N, 1:3]
@polyvar f[1:N, 1]
# print("fuck")

gravity = temp_p[1:3, 1] * 0
gravity[1] = 0.0
gravity[2] = 0.0
gravity[3] = 9.81

eq = [temp_p[1,1] * 0]
ineq = []
# print("fuck")

eq[1] = q[1, 1] - 1.0
append!(eq, [q[1, 2] - 0.0])
append!(eq, [q[1, 3] - 0.0])
append!(eq, [q[1, 4] - 0.0])

append!(eq, [w[1, 1] - 2 / dt])
append!(eq, [w[1, 2] - 0.0])
append!(eq, [w[1, 3] - 0.0])
append!(eq, [w[1, 4] - 0.0])

append!(eq, [p[1, 1] - 2])
append!(eq, [p[1, 2] - 0.0])
append!(eq, [p[1, 3] - 0.0])

append!(eq, [v[1, 1] - 0.0])
append!(eq, [v[1, 2] - 0.0])
append!(eq, [v[1, 3] - 0.0])

var = append!(reshape(q, 4 * (N+1)), reshape(w, 4*(N+1)), 
              reshape(p, 3 * (N+1)), reshape(v, 3*(N+1)), 
              reshape(tau, 3*N), reshape(f, 1 * N))
Cost = 0
for k = 1:N

    qk = q[k, 1:4]
    qkp1 = q[k+1, 1:4]

    wk = w[k, 1:4]
    wkp1 = w[k+1, 1:4]

    tauk = tau[k, 1:3]

    Mk = Jb * wk[1] * wk[2:4] + skew(wk[2:4]) * Jb * wk[2:4]
    Mkp1 = Jb * wkp1[1] * wkp1[2:4] + skew(wkp1[2:4]) * Jb * wkp1[2:4]

    append!(eq, Mkp1 - Mk - 2 * tauk)
    append!(eq, qkp1 - dt / 2 * LeftMultiplication(qk) * wk)

    append!(eq, [qkp1[1]^2 + qkp1[2]^2 + qkp1[3]^2 + qkp1[4]^2 - 1])
    append!(eq, [(wkp1[1]^2 + wkp1[2]^2 + wkp1[3]^2 + wkp1[4]^2) * dt^2 - 4])
    
    append!(ineq, [wkp1[1]])
    append!(ineq, [qkp1[1]])


    pk   = p[k, 1:3]
    pkp1 = p[k+1, 1:3]

    vk   = v[k, 1:3]
    vkp1 = v[k+1, 1:3]

    fk = G * f[k]
    print(fk)

    fbk = V * RightMultiplication(qkp1)' * LeftMultiplication(qkp1) * V' * fk
    print(fbk)
    append!(eq, m * (vkp1 - vk) / dt + m * gravity - fbk )
    append!(eq, pkp1 - (pk + vk * dt) )

    Cost = Cost + (tauk[1]^2 + tauk[2]^2 + tauk[3]^2) * 0.01
    Cost = Cost + ((wk[1] - 1)^2 + wk[2]^2 + wk[3]^2 + wk[4]^2)
    Cost = Cost + (qk[1]^2 + (qk[2] - 1)^2 + qk[3]^2 + qk[4]^2)

    Cost = Cost + (vk[1]^2 + vk[2]^2 + vk[3]^2) * 0.01
    Cost = Cost + (pk[1]^2 + pk[2]^2 + (pk[3] - 2)^2)

end

qkp1 = q[end, :]
wkp1 = w[end, :]
pkp1 = p[end, :]
vkp1 = v[end, :]

Cost = Cost + ((wkp1[1] - 1)^2 + wkp1[2]^2 + wkp1[3]^2 + wkp1[4]^2)
Cost = Cost + (qkp1[1]^2 + (qkp1[2] - 1)^2 + qkp1[3]^2 + qkp1[4]^2)

Cost = Cost + (vkp1[1]^2 + vkp1[2]^2 + vkp1[3]^2)
Cost = Cost + (pkp1[1]^2 + pkp1[2]^2 + (pkp1[3] - 2)^2)

pop = append!([Cost], ineq, eq)
eq_len = length(eq)

order = 1
# opt,sol,data =cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS = "block", solution=true, QUIET=false, MomentOne=false)
opt,sol,data =cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS = "MD" , solution=true, QUIET=false, MomentOne=false)

Polynomial{true, Float64}[0.0, 0.0, f₁₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₂₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₃₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₄₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₅₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₆₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₇₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₈₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₉₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₁₀₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₁₁₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}[0.0, 0.0, f₁₂₋₁]Polynomial{true, Float64}[0.0, 0.0, 0.0]Polynomial{true, Float64}

(10226.76005287912, [0.9999998885612813, 0.9999994321265072, 0.9673961237781952, 0.8762498307821722, 0.7585904391676039, 0.6381743516565868, 0.527200636988981, 0.43056307242435626, 0.3491679676430812, 0.2820092478635035  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], TSLTSSOS.mcpop_data(374, 0, 354, 314, Vector{Vector{UInt16}}[[[], [0x0001, 0x0001], [0x0002, 0x0002], [0x0003, 0x0003], [0x0004, 0x0004], [0x0005, 0x0005], [0x0006, 0x0006], [0x0007, 0x0007], [0x0008, 0x0008], [0x0009, 0x0009]  …  [0x0159, 0x0159], [0x015a, 0x015a], [0x015b, 0x015b], [0x015c, 0x015c], [0x015d, 0x015d], [0x015e, 0x015e], [0x015f, 0x015f], [0x0160, 0x0160], [0x0161, 0x0161], [0x0162, 0x0162]], [[0x0056]], [[0x0002]], [[0x0057]], [[0x0003]], [[0x0058]], [[0x0004]], [[0x0059]], [[0x0005]], [[0x005a]]  …  [[0x0014, 0x0092], [0x0029, 0x00a7], [0x003e, 0x0068], [0x0053, 0x007d], [0x003f]], [[0x0014, 0x00a7], [0x0029, 0x0092], [0x003e, 0x007d], [0x0053, 0x0068], [0x0054]], [[0x0015, 0x0015], [0x002a, 0x002a

In [66]:
k = 1
fk = G * f[k]
qkp1 = q[end, :]
# fbk = V * RightMultiplication(qkp1)' * LeftMultiplication(qkp1) * V' * fk
V

3×4 Matrix{Polynomial{true, Float64}}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0